In [3]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

furPattern_load = pd.read_csv("Tree_Data_Range_5000_CSV.csv",delimiter=",",header=None) # Load the entire CSV file.
x_furPattern_train = furPattern_load.values[:5000,6] # Takes the first 5,000 x-values for the breed. In "Column G" excel.
y_furPattern_train = furPattern_load.values[:5000,7] # Takes the first 5,000 y-values for the breed. In "Column H" excel.

#x_test = furPattern_load.values[:5000,6]
#y_test = furPattern_load.values[:5000,7]


# One hot encoding
dummy_y_train = np_utils.to_categorical(y_furPattern_train)
dummy_y_train = np.delete(dummy_y_train, obj=0,axis=1)

# Function that creates our neural network for the cat weights
def weights_NN():
    # Creating model
    model = Sequential()
   
    # Adding input layer with rectified linear activation function for input layer
    model.add(Dense(8, input_dim=1, activation='relu'))
    
    # Adding output layer with softmax activation function
    model.add(Dense(5, activation='softmax')) # Dimension here is 5 because there are 5 labels to map to.
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


estimator = KerasClassifier(build_fn=weights_NN, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, x_furPattern_train, dummy_y_train, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
# The above gets "Baseline: 98.42% (1.19%)"
'''
Without using estimator: 5,000 points used for training and the same 5,000 points used for testing.
The testAccuracy() function was used. This approach returns estimates of ~96.86%.

Without using estimator: the first 2,500 points used for training, and the last 2,500 points used for
testing. The testAccuracy() function was used. This approach returns estimates of ~98.68%.
'''

'''
model = weights_NN()
model.fit(x_furPattern_train, dummy_y_train, epochs=200, batch_size=15, verbose=0)

## Function for predicting labels using keras predict function
def predict(model,x):
    preds = np.argmax(model.predict(x), axis=-1)
    #preds = model.predict_classes(x)
    for i in range(len(preds)):
        preds[i] = preds[i] + 1
    return preds

## Function for testing accuracy of predictions
def testAccuracy(preds, labels):
    count = 0
    for i in range(len(preds)):
        if(preds[i] == labels[i]):
            count += 1
    final = 100*(count/len(labels))
    return print("Accuracy: %.2f" % (final))

predictions = predict(model,x_test)
testAccuracy(predictions,y_test)
'''

Baseline: 98.42% (1.19%)


'\nmodel = weights_NN()\nmodel.fit(x_furPattern_train, dummy_y_train, epochs=200, batch_size=15, verbose=0)\n\n## Function for predicting labels using keras predict function\ndef predict(model,x):\n    preds = np.argmax(model.predict(x), axis=-1)\n    #preds = model.predict_classes(x)\n    for i in range(len(preds)):\n        preds[i] = preds[i] + 1\n    return preds\n\n## Function for testing accuracy of predictions\ndef testAccuracy(preds, labels):\n    count = 0\n    for i in range(len(preds)):\n        if(preds[i] == labels[i]):\n            count += 1\n    final = 100*(count/len(labels))\n    return print("Accuracy: %.2f" % (final))\n\npredictions = predict(model,x_test)\ntestAccuracy(predictions,y_test)\n'